# Estrapolazione dati delle città dai file html
- Con l'ausilio della libreria spacy procederemo ad estrapolare i dati delle città dal 2013 in poi.

In [1]:
import spacy
import spacy_transformers
from spacy.language import Language
import pandas as pd
import pickle

In [2]:
#carico la lista coordinate city

infile = open("coordinate_city_list.pickle",'rb')
coordinate_city_list = pickle.load(infile)
infile.close()

In [3]:
coordinate_city_list

[['57.05N', '10.33E', 'Århus'],
 ['57.05N', '10.33E', 'Aalborg'],
 ['40.99N', '27.69E', 'Çorlu'],
 ['40.99N', '27.69E', 'Esenyurt'],
 ['40.99N', '27.69E', 'Tekirdag'],
 ['40.99N', '34.08E', 'Çorum'],
 ['50.63N', '82.39E', 'Öskemen'],
 ['44.20N', '87.20E', 'Ürümqi'],
 ['44.20N', '87.20E', 'Changji'],
 ['42.59N', '8.73W', 'A Coruña'],
 ['42.59N', '8.73W', 'Vigo'],
 ['50.63N', '6.34E', 'Aachen'],
 ['50.63N', '6.34E', 'Bergisch Gladbach'],
 ['50.63N', '6.34E', 'Bonn'],
 ['50.63N', '6.34E', 'Cologne'],
 ['50.63N', '6.34E', 'Düsseldorf'],
 ['50.63N', '6.34E', 'Duisburg'],
 ['50.63N', '6.34E', 'Essen'],
 ['50.63N', '6.34E', 'Koblenz'],
 ['50.63N', '6.34E', 'Krefeld'],
 ['50.63N', '6.34E', 'Leverkusen'],
 ['50.63N', '6.34E', 'Liège'],
 ['50.63N', '6.34E', 'Mönchengladbach'],
 ['50.63N', '6.34E', 'Mülheim'],
 ['50.63N', '6.34E', 'Maastricht'],
 ['50.63N', '6.34E', 'Moers'],
 ['50.63N', '6.34E', 'Neuss'],
 ['50.63N', '6.34E', 'Oberhausen'],
 ['50.63N', '6.34E', 'Remscheid'],
 ['50.63N', '6.34E',

In [64]:
#carico  la lista delle città con doppioni

infile = open("probl_cities.pickle",'rb')
probl_cities = pickle.load(infile)
infile.close()

In [65]:
probl_cities

[['31.35N', '30.16E', 'Alexandria'],
 ['39.38N', '76.99W', 'Alexandria'],
 ['39.38N', '76.99W', 'Arlington'],
 ['32.95N', '96.70W', 'Arlington'],
 ['39.38N', '104.05W', 'Aurora'],
 ['40.99N', '87.34W', 'Aurora'],
 ['40.99N', '2.13E', 'Barcelona'],
 ['10.45N', '64.64W', 'Barcelona'],
 ['26.52N', '77.01E', 'Bharatpur'],
 ['28.13N', '84.55E', 'Bharatpur'],
 ['52.24N', '2.63W', 'Birmingham'],
 ['32.95N', '87.13W', 'Birmingham'],
 ['52.24N', '23.65E', 'Brest'],
 ['49.03N', '4.90W', 'Brest'],
 ['18.48N', '96.34W', 'Córdoba'],
 ['37.78N', '4.07W', 'Córdoba'],
 ['36.17N', '5.97W', 'Cadiz'],
 ['10.45N', '123.55E', 'Cadiz'],
 ['52.24N', '0.00W', 'Cambridge'],
 ['42.59N', '72.00W', 'Cambridge'],
 ['10.45N', '76.09W', 'Cartagena'],
 ['37.78N', '0.00W', 'Cartagena'],
 ['24.92S', '49.66W', 'Colombo'],
 ['7.23N', '80.27E', 'Colombo'],
 ['32.95N', '85.21W', 'Columbus'],
 ['39.38N', '83.24W', 'Columbus'],
 ['7.23S', '109.46E', 'Depok'],
 ['5.63S', '106.55E', 'Depok'],
 ['7.23N', '4.05E', 'Ede'],
 ['52.

In [4]:
# carico pipeline spacy disattivando alcune funzioni per diminuire il tempo di esecuzione

nlp = spacy.load("en_core_web_sm", disable=['tensorizer','similarity','ner','tagger','textcat','sbd','merge_noun_chunks','merge_entities','lemmatizer'])

In [5]:
#aggiungo pipeline, voglio che si vada a capo quando si incontra \n 

@Language.component("set_custom_boundaries")
def set_custom_boundaries(doc):
    for token in doc[:-1]:
        if token.text == '\n  ':
            doc[token.i].is_sent_start = True
    return doc

nlp.add_pipe("set_custom_boundaries", before="parser")

<function __main__.set_custom_boundaries(doc)>

In [21]:
def get_data(nation):
    
    f=open(r'C:\Users\paolo\OneDrive\Desktop\progetto_python\data_harvesting\data_cities\GlobalTemp_city_dataset\{}.html'.format(nation))
    j=0
    a=f.read()
    clean_data_list=[]
    doc = nlp(a)
    control_list=[]
    l=[]
    reference_temp=[]
    reference_temp_2=[]
    clean_data_list_2=[]
    
    for token in doc:
        
        l.append(token.text)
  
    try:
        for n in range(len(l)):

            control = [l[n],l[n+1],l[n+2],l[n+3]]

            if control == ['Dec','\n','%','%']:
                
                reference_temp=l[n+4:n+28]

                break
                
    except IndexError:
        
            for n in range(len(l)):

                control = [l[n],l[n+1],l[n+2]]

                if control == ['Dec','\n','%']:
                    
                    reference_temp=l[n+3:n+27]

                    break
                    
                    
    while len(reference_temp_2)<12:
        
        if reference_temp[j] not in {' '*k for k in range(10)} :
            
            reference_temp_2.append(reference_temp[j])
            
        j+=1
            
                
    data_sent=list(doc.sents)[len(list(doc.sents))-110:]    
    data_list=[]
    
    
    for n,sent in enumerate(data_sent):
                
        try:
            
            if str(sent).split()[0] == '2013':

                data_sent=data_sent[n:]

                break
                
        except IndexError:
            
            continue
    
    for i,data in enumerate(data_sent):

        data_list.append(str(data).split())
        
        if len(data_list[i])>12:
        
            for item in data_list[i]:

                if item in ['</pre></body></html>','</pre></body></html', '>'] :

                    data_list[i].remove(item) 
                    
    for llist in data_list:
        
        if llist == ['>'] or llist==[]:
            
            data_list.remove(llist)
            
    for k,llist in enumerate(data_list):
        
        if len(llist)<12 and llist not in control_list :
            
            control_list.append(llist)
            control_list.append(data_list[k+1])            
            clean_data_list.append(llist+data_list[k+1])
            
        elif data_list[k] in clean_data_list:
            
            continue
            
        elif llist in control_list:
            
            continue
        
        else:
            
            clean_data_list.append(data_list[k])
    
            
    for h,llist in enumerate(clean_data_list):
        
        if len(llist)>12 and int(llist[0]) in range(1,13):
            
            correction=[]
            
            if llist[0] != '1':
                
                #correction = [clean_data_list[h-1][0]]+llist[0:11]
                
                llist.insert(0,clean_data_list[h-1][0])
                
            else:
                
                #correction = [clean_data_list[h+1][0]]+llist[0:11]
                
                llist.insert(0,clean_data_list[h+1][0])
                
            clean_data_list_2.append(llist[0:12])
            clean_data_list_2.append(llist[12:24])
            
        else:
            
            clean_data_list_2.append(llist)
        
    for h,llist in enumerate(clean_data_list_2):

        if int(llist[0]) not in list(range(2013,2021)):

            if str(llist[11]) != 'NaN':

                if int(llist[11]) in list(range(2013,2021)):

                    v=llist[11]
                    llist.pop()

                    llist.insert(0,v)

            elif int(llist[0]) in list(range(1,13)) and int(llist[1]) in list(range(1,13)):

                if llist[1] != '1':

                    llist[0]=clean_data_list[h-1][0]

                else:

                    llist[0]=str(int(clean_data_list[h-1][0])+1)        
     
        
    return clean_data_list_2, list(map(float,reference_temp_2))

In [8]:
def get_temp(data_list,reference_temp):
    
    temp_list=[]
    
    for data in data_list:
        
        temp_list.append( [data[0], data[1], float(data[2])+reference_temp[int(data[1])-1]] )
    
    return temp_list

In [33]:
#variante della funzione get_data da utilizzare sulle città con doppioni

def get_data_2(coord):
    
    f=open(r'C:\Users\paolo\OneDrive\Desktop\progetto_python\data_harvesting\data_cities\GlobalTemp_city_dataset_correction\{}{}{}.html'.format(coord[2],coord[0],coord[1]))
    j=0
    a=f.read()
    clean_data_list=[]
    doc = nlp(a)
    control_list=[]
    l=[]
    
    reference_temp=[]
    reference_temp_2=[]
    clean_data_list_2=[]
    
    for token in doc:
        
        l.append(token.text)
  
    try:
        for n in range(len(l)):

            control = [l[n],l[n+1],l[n+2],l[n+3]]

            if control == ['Dec','\n','%','%']:
                
                reference_temp=l[n+4:n+28]

                #for j in range(0,24,2):

                    #reference_temp.append(l[n+5+j])

                break
                
    except IndexError:
        
            for n in range(len(l)):

                control = [l[n],l[n+1],l[n+2]]

                if control == ['Dec','\n','%']:
                    
                    reference_temp=l[n+3:n+27]

                    #for j in range(0,24,2):

                     #   reference_temp.append(l[n+4+j])

                    break
                    
                    
    while len(reference_temp_2)<12:
        
        if reference_temp[j] not in {' '*k for k in range(10)} :
            
            reference_temp_2.append(reference_temp[j])
            
        j+=1
            
                
    data_sent=list(doc.sents)[len(list(doc.sents))-110:]    
    data_list=[]
    
    
    for n,sent in enumerate(data_sent):
                
        try:
            
            if str(sent).split()[0] == '2013':

                data_sent=data_sent[n:]

                break
                
        except IndexError:
            
            continue
    
    for i,data in enumerate(data_sent):

        data_list.append(str(data).split())
        
        if len(data_list[i])>12:
        
            for item in data_list[i]:

                if item in ['</pre></body></html>','</pre></body></html', '>'] :

                    data_list[i].remove(item) 
                    
    for llist in data_list:
        
        if llist == ['>'] or llist==[]:
            
            data_list.remove(llist)
            
    for k,llist in enumerate(data_list):
        
        if len(llist)<12 and llist not in control_list :
            
            control_list.append(llist)
            control_list.append(data_list[k+1])
            
            #data_list[k+1].insert(0,llist[0])
            
            clean_data_list.append(llist+data_list[k+1])
            
        elif data_list[k] in clean_data_list:
            
            continue
            
        elif llist in control_list:
            
            continue
        
        else:
            
            clean_data_list.append(data_list[k])
            
            
    
    
            
    for h,llist in enumerate(clean_data_list):
        
        if len(llist)>12 and int(llist[0]) in range(1,13):
            
            correction=[]
            
            if llist[0] != '1':
                
                #correction = [clean_data_list[h-1][0]]+llist[0:11]
                
                llist.insert(0,clean_data_list[h-1][0])
                
            else:
                
                #correction = [clean_data_list[h+1][0]]+llist[0:11]
                
                llist.insert(0,clean_data_list[h+1][0])
                
            clean_data_list_2.append(llist[0:12])
            clean_data_list_2.append(llist[12:24])
            
        else:
            
            clean_data_list_2.append(llist)
    
    
    for h,llist in enumerate(clean_data_list_2):

        if int(llist[0]) not in list(range(2013,2021)):

            if str(llist[11]) != 'NaN':

                if int(llist[11]) in list(range(2013,2021)):

                    v=llist[11]
                    llist.pop()

                    llist.insert(0,v)

            elif int(llist[0]) in list(range(1,13)) and int(llist[1]) in list(range(1,13)):

                if llist[1] != '1':

                    llist[0]=clean_data_list[h-1][0]

                else:

                    llist[0]=str(int(clean_data_list[h-1][0])+1)        
     
        
    return clean_data_list_2, list(map(float,reference_temp_2))
        

In [15]:
#costruisco dataframe aggiornato delle città che mi hanno dato errori

dt=[]
temp_list=[]
c_list=[]
c_errors_2=[]
dict_city={}
coordinate_control_set=set()
latitude=[]
longitude=[]

occurence_control_list=[]

for coordinate in probl_cities:
    
    if coordinate in occurence_control_list:
        
        continue
    
    
    elif (coordinate[0],coordinate[1]) in coordinate_control_set:
        
        for item in dict_city[(coordinate[0],coordinate[1])]:

            if int(item[1])<10:

                dt.append(item[0]+'-0'+item[1]+'-01')
                temp_list.append(item[2])
                c_list.append(coordinate[2])
                latitude.append(coordinate[0])
                longitude.append(coordinate[1])
                
            else:

                dt.append(item[0]+'-'+item[1]+'-01')
                temp_list.append(item[2])
                c_list.append(coordinate[2])
                latitude.append(coordinate[0])
                longitude.append(coordinate[1])
                
        occurence_control_list.append(coordinate)
                
                
    else:

        try:
            (a,b) = get_data_2(coordinate)
            
            temp_data=get_temp(a,b)

            for item in temp_data:

                if int(item[1])<10:

                    dt.append(item[0]+'-0'+item[1]+'-01')
                    temp_list.append(item[2])
                    c_list.append(coordinate[2])
                    latitude.append(coordinate[0])
                    longitude.append(coordinate[1])

                else:

                    dt.append(item[0]+'-'+item[1]+'-01')
                    temp_list.append(item[2])
                    c_list.append(coordinate[2])
                    latitude.append(coordinate[0])
                    longitude.append(coordinate[1])

            dict_city[(coordinate[0],coordinate[1])]=temp_data
            
            coordinate_control_set.add((coordinate[0],coordinate[1]))
            
            occurence_control_list.append(coordinate)
            
            print(len(coordinate_control_set), end=' - ')
            


        except:

            c_errors_2.append(coordinate)
            print(coordinate[2], end=' - ')
        
            
        
dataset_dict_c={'dt':dt,'AverageTemperature':temp_list,'City':c_list,'Latitude':latitude,'Longitude':longitude}

df_update_d=pd.DataFrame.from_dict(dataset_dict_c)

1 - 2 - 3 - 4 - 5 - 6 - 7 - 8 - 9 - 10 - 11 - 12 - 13 - 14 - 15 - 16 - 17 - 18 - 19 - 20 - 21 - 22 - 23 - 24 - 25 - 26 - 27 - 28 - 29 - 30 - 31 - 32 - 33 - 34 - 35 - 36 - 37 - 38 - 39 - 40 - 41 - 42 - 43 - 44 - 45 - 46 - 47 - 48 - 49 - 50 - 51 - 52 - 53 - 54 - 55 - 56 - 57 - 58 - 59 - 60 - 61 - 62 - 63 - 64 - 65 - 66 - 67 - 68 - 69 - 70 - 71 - 72 - 73 - 74 - 75 - 76 - 77 - 78 - 79 - 80 - 81 - 82 - 83 - 84 - 85 - 86 - 87 - 88 - 89 - 90 - 91 - 92 - 93 - 94 - 95 - 96 - 97 - 98 - 99 - 100 - 

In [68]:
df_update_d

,dt,AverageTemperature,City,Latitude,Longitude
0,2013-01-01,14.757,Alexandria,31.35N,30.16E
1,2013-02-01,15.766,Alexandria,31.35N,30.16E
2,2013-03-01,18.234,Alexandria,31.35N,30.16E
3,2013-04-01,18.974,Alexandria,31.35N,30.16E
4,2013-05-01,23.425,Alexandria,31.35N,30.16E
...,...,...,...,...,...
11419,2020-08-01,29.478,Yingcheng,24.92N,113.50E
11420,2020-09-01,26.002,Yingcheng,24.92N,113.50E
11421,2020-10-01,21.259,Yingcheng,24.92N,113.50E
11422,2020-11-01,18.703,Yingcheng,24.92N,113.50E


In [17]:
probl_city_set=set()

for city in probl_cities:
    
    probl_city_set.add(city[2])

In [18]:
probl_city_set

{'Alexandria',
 'Arlington',
 'Aurora',
 'Barcelona',
 'Bharatpur',
 'Birmingham',
 'Brest',
 'Cadiz',
 'Cambridge',
 'Cartagena',
 'Colombo',
 'Columbus',
 'Córdoba',
 'Depok',
 'Ede',
 'Glendale',
 'Haicheng',
 'Haikou',
 'Hamilton',
 'Hyderabad',
 'Jamalpur',
 'Jining',
 'Kadoma',
 'Kingston',
 'La Paz',
 'Lalitpur',
 'León',
 'London',
 'Los Angeles',
 'Luoyang',
 'Manchester',
 'Manzanillo',
 'Newcastle',
 'Pasadena',
 'Peoria',
 'Richmond',
 'Rongcheng',
 'Saint Louis',
 'Saint Petersburg',
 'Salamanca',
 'San Cristóbal',
 'San Juan',
 'Santa Ana',
 'Santa Clara',
 'Santa Rosa',
 'Santiago',
 'Santo Domingo',
 'Springfield',
 'Suzhou',
 'Syracuse',
 'Taman',
 'Toledo',
 'Valencia',
 'Vancouver',
 'Worcester',
 'Yichun',
 'Yingcheng'}

In [22]:
#costruisco dataframe aggiornato

dt=[]
temp_list=[]
c_list=[]
c_errors=[]
latitude=[]
longitude=[]
dict_city={}
coordinate_control_set=set()

for coordinate in coordinate_city_list:
    
    if coordinate[2] in probl_city_set:
        
        continue
    
    
    elif (coordinate[0],coordinate[1]) in coordinate_control_set:
        
        for item in dict_city[(coordinate[0],coordinate[1])]:

            if int(item[1])<10:

                dt.append(item[0]+'-0'+item[1]+'-01')
                temp_list.append(item[2])
                c_list.append(coordinate[2])
                latitude.append(coordinate[0])
                longitude.append(coordinate[1])
                
                
            else:

                dt.append(item[0]+'-'+item[1]+'-01')
                temp_list.append(item[2])
                c_list.append(coordinate[2])
                latitude.append(coordinate[0])
                longitude.append(coordinate[1])
                
                
    else:

        try:
            (a,b) = get_data(coordinate[2])
            temp_data=get_temp(a,b)

            for item in temp_data:

                if int(item[1])<10:

                    dt.append(item[0]+'-0'+item[1]+'-01')
                    temp_list.append(item[2])
                    c_list.append(coordinate[2])
                    latitude.append(coordinate[0])
                    longitude.append(coordinate[1])

                else:

                    dt.append(item[0]+'-'+item[1]+'-01')
                    temp_list.append(item[2])
                    c_list.append(coordinate[2])
                    latitude.append(coordinate[0])
                    longitude.append(coordinate[1])

            dict_city[(coordinate[0],coordinate[1])]=temp_data
            
            coordinate_control_set.add((coordinate[0],coordinate[1]))
            
            print(len(coordinate_control_set), end=' - ')
            


        except:

            c_errors.append(coordinate)
        
            
        
dataset_dict={'dt':dt,'AverageTemperature':temp_list,'City':c_list,'Latitude':latitude,'Longitude':longitude}

df_update=pd.DataFrame.from_dict(dataset_dict)

1 - 2 - 3 - 4 - 5 - 6 - 7 - 8 - 9 - 10 - 11 - 12 - 13 - 14 - 15 - 16 - 17 - 18 - 19 - 20 - 21 - 22 - 23 - 24 - 25 - 26 - 27 - 28 - 29 - 30 - 31 - 32 - 33 - 34 - 35 - 36 - 37 - 38 - 39 - 40 - 41 - 42 - 43 - 44 - 45 - 46 - 47 - 48 - 49 - 50 - 51 - 52 - 53 - 54 - 55 - 56 - 57 - 58 - 59 - 60 - 61 - 62 - 63 - 64 - 65 - 66 - 67 - 68 - 69 - 70 - 71 - 72 - 73 - 74 - 75 - 76 - 77 - 78 - 79 - 80 - 81 - 82 - 83 - 84 - 85 - 86 - 87 - 88 - 89 - 90 - 91 - 92 - 93 - 94 - 95 - 96 - 97 - 98 - 99 - 100 - 101 - 102 - 103 - 104 - 105 - 106 - 107 - 108 - 109 - 110 - 111 - 112 - 113 - 114 - 115 - 116 - 117 - 118 - 119 - 120 - 121 - 122 - 123 - 124 - 125 - 126 - 127 - 128 - 129 - 130 - 131 - 132 - 133 - 134 - 135 - 136 - 137 - 138 - 139 - 140 - 141 - 142 - 143 - 144 - 145 - 146 - 147 - 148 - 149 - 150 - 151 - 152 - 153 - 154 - 155 - 156 - 157 - 158 - 159 - 160 - 161 - 162 - 163 - 164 - 165 - 166 - 167 - 168 - 169 - 170 - 171 - 172 - 173 - 174 - 175 - 176 - 177 - 178 - 179 - 180 - 181 - 182 - 183 - 184 - 185 

1330 - 1331 - 1332 - 1333 - 1334 - 1335 - 1336 - 1337 - 1338 - 1339 - 1340 - 1341 - 

In [23]:
c_errors

[['10.45S', '48.27W', 'Palmas']]

In [34]:
#devo ricavare i dati aggiornati di Palmas

(a,b)=get_data('Palmas')

ValueError: invalid literal for int() with base 10: '1.164'

In [38]:
#definisco variante get_data per risolvere il problema della città Palmas

def get_data_p(nation):
    
    f=open(r'C:\Users\paolo\OneDrive\Desktop\progetto_python\data_harvesting\data_cities\GlobalTemp_city_dataset\{}.html'.format(nation))
    j=0
    a=f.read()
    clean_data_list=[]
    doc = nlp(a)
    control_list=[]
    l=[]
    
    reference_temp=[]
    reference_temp_2=[]
    clean_data_list_2=[]
    
    for token in doc:
        
        l.append(token.text)
  
    try:
        for n in range(len(l)):

            control = [l[n],l[n+1],l[n+2],l[n+3]]

            if control == ['Dec','\n','%','%']:
                
                reference_temp=l[n+4:n+28]

                break
                
    except IndexError:
        
            for n in range(len(l)):

                control = [l[n],l[n+1],l[n+2]]

                if control == ['Dec','\n','%']:
                    
                    reference_temp=l[n+3:n+27]

                    break
                    
                    
    while len(reference_temp_2)<12:
        
        if reference_temp[j] not in {' '*k for k in range(10)} :
            
            reference_temp_2.append(reference_temp[j])
            
        j+=1
            
                
    data_sent=list(doc.sents)[len(list(doc.sents))-110:]    
    data_list=[]
    
    
    for n,sent in enumerate(data_sent):
                
        try:
            
            if str(sent).split()[0] == '2013':

                data_sent=data_sent[n:]

                break
                
        except IndexError:
            
            continue
    
    for i,data in enumerate(data_sent):

        data_list.append(str(data).split())
        
        if len(data_list[i])>12:
        
            for item in data_list[i]:

                if item in ['</pre></body></html>','</pre></body></html', '>'] :

                    data_list[i].remove(item) 
                    
    for llist in data_list:
        
        if llist == ['>'] or llist==[]:
            
            data_list.remove(llist)
            
    for k,llist in enumerate(data_list):
        
        if len(llist)<12 and llist not in control_list :
            
            control_list.append(llist)
            control_list.append(data_list[k+1])            
            clean_data_list.append(llist+data_list[k+1])
            
        elif data_list[k] in clean_data_list:
            
            continue
            
        elif llist in control_list:
            
            continue
        
        else:
            
            clean_data_list.append(data_list[k])
            
    try:
            
        for h,llist in enumerate(clean_data_list):

            if len(llist)>12 and int(llist[0]) in range(1,13):

                correction=[]

                if llist[0] != '1':

                    llist.insert(0,clean_data_list[h-1][0])

                else:

                    llist.insert(0,clean_data_list[h+1][0])

                clean_data_list_2.append(llist[0:12])
                clean_data_list_2.append(llist[12:24])

            else:

                clean_data_list_2.append(llist)


        for h,llist in enumerate(clean_data_list_2):

            if int(llist[0]) not in list(range(2013,2021)):

                if str(llist[11]) != 'NaN':

                    if int(llist[11]) in list(range(2013,2021)):

                        v=llist[11]
                        llist.pop()
                        llist.insert(0,v)

                elif int(llist[0]) in list(range(1,13)) and int(llist[1]) in list(range(1,13)):

                    if llist[1] != '1':

                        llist[0]=clean_data_list[h-1][0]

                    else:

                        llist[0]=str(int(clean_data_list[h-1][0])+1)
    except ValueError:
        
        clean_data_list_2=clean_data_list
     
        
    return clean_data_list_2, list(map(float,reference_temp_2))

In [39]:
(a,b)=get_data_p('Palmas')

In [40]:
a

[['2013',
  '1',
  '0.746',
  '0.465',
  '1.151',
  '0.085',
  '0.964',
  '0.104',
  '1.145',
  '0.084',
  'NaN',
  'NaN'],
 ['2013',
  '2',
  '1.301',
  '0.560',
  '1.175',
  '0.080',
  '0.959',
  '0.102',
  '1.148',
  '0.084',
  'NaN',
  'NaN'],
 ['2013',
  '3',
  '1.000',
  '0.419',
  '1.159',
  '0.142',
  '0.964',
  '0.100',
  '1.154',
  '0.083',
  'NaN',
  'NaN'],
 ['2013',
  '4',
  '0.896',
  '0.522',
  '1.112',
  '0.127',
  '0.979',
  '0.107',
  '1.157',
  '0.083',
  'NaN',
  'NaN'],
 ['2013',
  '5',
  '0.737',
  '0.607',
  '1.108',
  '0.117',
  '1.003',
  '0.112',
  '1.165',
  '0.083',
  'NaN',
  'NaN'],
 ['2013',
  '6',
  '1.292',
  '0.600',
  '0.973',
  '0.206',
  '1.028',
  '0.114',
  '1.169',
  '0.084',
  'NaN',
  'NaN'],
 ['2013',
  '7',
  '1.814',
  '0.438',
  '0.927',
  '0.210',
  '1.032',
  '0.112',
  '1.173',
  '0.085',
  'NaN',
  'NaN'],
 ['2013',
  '8',
  '0.899',
  '0.547',
  '0.822',
  '0.273',
  '1.079',
  '0.108',
  '1.180',
  '0.089',
  'NaN',
  'NaN'],
 ['2013'

In [41]:
#sistemazione relativa a Palmas

a_1=[]

for i,item in enumerate(a):

    if len(item)<12:
        
        b_1 = a[i]+a[i+1]
        
        c_1=b_1[0:12]
        c_2=b_1[12:24]
        
        a_1.append(c_1)
        a_1.append(c_2)
        
        
    elif len(item)>12:
        
        continue
    
    else:
        a_1.append(item)
        

In [42]:
a_1

[['2013',
  '1',
  '0.746',
  '0.465',
  '1.151',
  '0.085',
  '0.964',
  '0.104',
  '1.145',
  '0.084',
  'NaN',
  'NaN'],
 ['2013',
  '2',
  '1.301',
  '0.560',
  '1.175',
  '0.080',
  '0.959',
  '0.102',
  '1.148',
  '0.084',
  'NaN',
  'NaN'],
 ['2013',
  '3',
  '1.000',
  '0.419',
  '1.159',
  '0.142',
  '0.964',
  '0.100',
  '1.154',
  '0.083',
  'NaN',
  'NaN'],
 ['2013',
  '4',
  '0.896',
  '0.522',
  '1.112',
  '0.127',
  '0.979',
  '0.107',
  '1.157',
  '0.083',
  'NaN',
  'NaN'],
 ['2013',
  '5',
  '0.737',
  '0.607',
  '1.108',
  '0.117',
  '1.003',
  '0.112',
  '1.165',
  '0.083',
  'NaN',
  'NaN'],
 ['2013',
  '6',
  '1.292',
  '0.600',
  '0.973',
  '0.206',
  '1.028',
  '0.114',
  '1.169',
  '0.084',
  'NaN',
  'NaN'],
 ['2013',
  '7',
  '1.814',
  '0.438',
  '0.927',
  '0.210',
  '1.032',
  '0.112',
  '1.173',
  '0.085',
  'NaN',
  'NaN'],
 ['2013',
  '8',
  '0.899',
  '0.547',
  '0.822',
  '0.273',
  '1.079',
  '0.108',
  '1.180',
  '0.089',
  'NaN',
  'NaN'],
 ['2013'

In [43]:
b

[25.71,
 25.65,
 25.9,
 26.14,
 26.18,
 25.47,
 25.25,
 26.68,
 28.04,
 27.17,
 26.43,
 25.75]

In [44]:
temp_data_palmas = get_temp(a_1,b)

In [45]:
temp_data_palmas

[['2013', '1', 26.456],
 ['2013', '2', 26.950999999999997],
 ['2013', '3', 26.9],
 ['2013', '4', 27.036],
 ['2013', '5', 26.916999999999998],
 ['2013', '6', 26.762],
 ['2013', '7', 27.064],
 ['2013', '8', 27.579],
 ['2013', '9', 29.349999999999998],
 ['2013', '10', 28.332],
 ['2013', '11', 26.975],
 ['2013', '12', 25.729],
 ['2014', '1', 25.902],
 ['2014', '2', 25.689],
 ['2014', '3', 26.078],
 ['2014', '4', 26.558],
 ['2014', '5', 27.1],
 ['2014', '6', 27.06],
 ['2014', '7', 26.98],
 ['2014', '8', 27.68],
 ['2014', '9', 29.224],
 ['2014', '10', 28.657000000000004],
 ['2014', '11', 27.706],
 ['2014', '12', 26.463],
 ['2015', '1', 27.159000000000002],
 ['2015', '2', 26.314],
 ['2015', '3', 26.421],
 ['2015', '4', 26.400000000000002],
 ['2015', '5', 27.084],
 ['2015', '6', 27.267],
 ['2015', '7', 27.0],
 ['2015', '8', 27.194],
 ['2015', '9', 30.029999999999998],
 ['2015', '10', 29.637],
 ['2015', '11', 28.919],
 ['2015', '12', 28.3],
 ['2016', '1', 26.295],
 ['2016', '2', 28.622999999999

In [71]:
c_list_p=[]
temp_list_p=[]
dt_p=[]
latitude_p=[]
longitude_p=[]
coordinate_palmas=['10.45S','48.27W']

    
for item in temp_data_palmas:

    if int(item[1])<10:

        dt_p.append(item[0]+'-0'+item[1]+'-01')
        temp_list_p.append(item[2])
        c_list_p.append('Palmas')
        latitude_p.append(coordinate_palmas[0])
        longitude_p.append(coordinate_palmas[1])

    else:

        dt_p.append(item[0]+'-'+item[1]+'-01')
        temp_list_p.append(item[2])
        c_list_p.append('Palmas')
        latitude_p.append(coordinate_palmas[0])
        longitude_p.append(coordinate_palmas[1])

    
dataset_dict_p={'dt':dt_p,'AverageTemperature':temp_list_p,'City':c_list_p, 'Latitude':latitude_p,'Longitude':longitude_p}

df_update_p=pd.DataFrame.from_dict(dataset_dict_p)

In [72]:
df_update_p

,dt,AverageTemperature,City,Latitude,Longitude
0,2013-01-01,26.456,Palmas,10.45S,48.27W
1,2013-02-01,26.951,Palmas,10.45S,48.27W
2,2013-03-01,26.900,Palmas,10.45S,48.27W
3,2013-04-01,27.036,Palmas,10.45S,48.27W
4,2013-05-01,26.917,Palmas,10.45S,48.27W
...,...,...,...,...,...
91,2020-08-01,27.905,Palmas,10.45S,48.27W
92,2020-09-01,29.548,Palmas,10.45S,48.27W
93,2020-10-01,29.012,Palmas,10.45S,48.27W
94,2020-11-01,27.360,Palmas,10.45S,48.27W


In [55]:
df_update

,dt,AverageTemperature,City,Latitude,Longitude
0,2013-01-01,-0.390,Århus,57.05N,10.33E
1,2013-02-01,-0.403,Århus,57.05N,10.33E
2,2013-03-01,-0.468,Århus,57.05N,10.33E
3,2013-04-01,5.393,Århus,57.05N,10.33E
4,2013-05-01,12.744,Århus,57.05N,10.33E
...,...,...,...,...,...
325435,2020-08-01,27.425,Zunyi,28.13N,106.36E
325436,2020-09-01,21.166,Zunyi,28.13N,106.36E
325437,2020-10-01,15.578,Zunyi,28.13N,106.36E
325438,2020-11-01,13.323,Zunyi,28.13N,106.36E


In [73]:
frames=[df_update,df_update_d,df_update_p]

df_cities_updated=pd.concat(frames)
df_cities_updated

,dt,AverageTemperature,City,Latitude,Longitude
0,2013-01-01,-0.390,Århus,57.05N,10.33E
1,2013-02-01,-0.403,Århus,57.05N,10.33E
2,2013-03-01,-0.468,Århus,57.05N,10.33E
3,2013-04-01,5.393,Århus,57.05N,10.33E
4,2013-05-01,12.744,Århus,57.05N,10.33E
...,...,...,...,...,...
91,2020-08-01,27.905,Palmas,10.45S,48.27W
92,2020-09-01,29.548,Palmas,10.45S,48.27W
93,2020-10-01,29.012,Palmas,10.45S,48.27W
94,2020-11-01,27.360,Palmas,10.45S,48.27W


In [75]:
df_cities_updated.to_csv(r'C:\Users\paolo\OneDrive\Desktop\progetto_python\data_harvesting\dataset_citta_aggiornamento_2020.csv')